# Load/save and structure data

You will learn to **load and save data** both to and from offline sources (e.g. CSV or Excel). You will learn about **pandas series and dataframes**, and how to clean, rename, structure and index your data.

**Links to more information:**

1. Official [tutorials](https://pandas.pydata.org/pandas-docs/stable/getting_started/tutorials.html)
2. DataCamp's [pandas' cheat sheet](https://www.datacamp.com/community/blog/python-pandas-cheat-sheet)

# Pandas data frames

In [1]:
import pandas as pd

In Pandas, the fundamental object of interest is a pandas dataframe. For example, 

In [2]:
X = pd.DataFrame(data = [[1,11.7,'Vitus'],[2,13.9,'Maximilian'],[3,14.6,'Bo-Bob']], columns=['id','inc','name'])
X

,id,inc,name
0,1,11.7,Vitus
1,2,13.9,Maximilian
2,3,14.6,Bo-Bob


A dataframe is essentially a matrix. 
* Rows = observations 
* Columns = variables 

In [3]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
id      3 non-null int64
inc     3 non-null float64
name    3 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 152.0+ bytes


## Indexing ("subsetting") 

Choosing a subset of the rows and/or columns of a dataframe is known as "indexing". All pandas dataframes are born with the method `loc[]`. 
* `df.loc[:, ['year']]` selects all rows (indicated by `:`) but only the column (variable) `year`. 
    * *Alternatively*,  `df['year']`
* `df.loc[df['year'] == 2002, :]` selects the rows where the variable `year` is equal to 2002 and all columns (indicated by `:`)
    * *Alternatively*,  `df[df['year'] == 2002]`
* `df.loc[df['year'] == 2002, ['name']]` selects the variable `name` and shows the rows where `year` is equal to 2002. 
    * *Alternatively*, `df[df['year']==2002]['name']`
    
In general, the syntax is `df.loc[CONDITION, [VARLIST]]`, where `CONDITION` is a vector of logical statements with the same length as the number of rows in the dataframe. 

**General rule:** Always use `df.loc` for assignments; you can use the alternatives if you are merely looking/plotting/etc.

In [4]:
X.loc[X['id'] > 1, ['name']]

,name
1,Maximilian
2,Bo-Bob


## Adding a variable

Variables are added with `df['newvar'] = SOMETHING`.

In [5]:
X['year'] = [2003, 2005, 2010]
X

,id,inc,name,year
0,1,11.7,Vitus,2003
1,2,13.9,Maximilian,2005
2,3,14.6,Bo-Bob,2010


**Note:** You cannot write `df.newvar = SOMETHING`. Some of you will forget. I promise. 

The something can be an expression based on other variables.

In [6]:
X['inc_adj'] = X['inc'] + X['year']/2000
X

,id,inc,name,year,inc_adj
0,1,11.7,Vitus,2003,12.7015
1,2,13.9,Maximilian,2005,14.9025
2,3,14.6,Bo-Bob,2010,15.6050


## The index

The first column in the dataset is referred to as the `index` of the dataframe. If you haven't done anything, it is just `[0, 1, 2, ....]`. If you specify integers instead of a vector of logicals into `df.loc[]` then pandas will find the rows where the index is equal to the values that you specify.

In [7]:
X

,id,inc,name,year,inc_adj
0,1,11.7,Vitus,2003,12.7015
1,2,13.9,Maximilian,2005,14.9025
2,3,14.6,Bo-Bob,2010,15.6050


In [8]:
X.loc[1, :]

id                  2
inc              13.9
name       Maximilian
year             2005
inc_adj       14.9025
Name: 1, dtype: object

In general, we do not want to look up using the index. If you want to see the top, use `df.head()`. 

In [9]:
X.head(2)

,id,inc,name,year,inc_adj
0,1,11.7,Vitus,2003,12.7015
1,2,13.9,Maximilian,2005,14.9025


## Assigning a different index

You can use many other things as indexes. This can sometimes be useful, but it is not necessary. 

For example the name: 

In [10]:
Y = X.set_index('name') # returns a copy
Y # notice how the layout changes

,id,inc,year,inc_adj
name,,,,
Vitus,1,11.7,2003,12.7015
Maximilian,2,13.9,2005,14.9025
Bo-Bob,3,14.6,2010,15.6050


In [11]:
Y.loc['Vitus']

id            1.0000
inc          11.7000
year       2003.0000
inc_adj      12.7015
Name: Vitus, dtype: float64

## Assignments to a subset of rows

Use a logical statement to select a subset of rows. Your RHS must then either be 
* A single value (all rows are set to this) 
* A list of values with same length as the number of selected rows.  

In [12]:
X

,id,inc,name,year,inc_adj
0,1,11.7,Vitus,2003,12.7015
1,2,13.9,Maximilian,2005,14.9025
2,3,14.6,Bo-Bob,2010,15.6050


In [13]:
Y = X.copy()

Y.loc[Y['id'] <= 2, ['name']] = 'hey'
Y

,id,inc,name,year,inc_adj
0,1,11.7,hey,2003,12.7015
1,2,13.9,hey,2005,14.9025
2,3,14.6,Bo-Bob,2010,15.6050


In [14]:
Y = X.copy()

Y.loc[(Y['name'] == 'Vitus') | (Y['year'] == 2005), ['name']] = ['Bib', 'Peter']
Y

,id,inc,name,year,inc_adj
0,1,11.7,Bib,2003,12.7015
1,2,13.9,Peter,2005,14.9025
2,3,14.6,Bo-Bob,2010,15.6050


## Data types

When you select an individual variable, it has the data type `series`. Some functions work on a pandas series (e.g. most numpy functions), but some only work on numpy arrays (e.g. scipy.stats.kurtosis). Therefore, it is nice to extract the underlying numpy objects: 
* `df`: pandas dataframe 
* `df['variable']`: pandas series 
* `df['variabe'].values`: Numpy array

In [15]:
type(X) 

pandas.core.frame.DataFrame

In [16]:
type(X['year'])

pandas.core.series.Series

In [17]:
type(X['year'].values)

numpy.ndarray

# Reading and writing data

In [18]:
# next, we need to make sure that we have the "data" subfolder and that it has the datasets we need 
# print contents of the "data" subfolder
import os 
os.listdir('./data')

['area.csv',
 'areal.xlsx',
 'INDKP107.xlsx',
 'INDKP107_tall.csv',
 'RAS200.xlsx',
 'RAS200_tall.csv']

## Reading in data

Pandas offers a lot of facilities for reading and writing to different formats. The functions have logical names: 
* CSV: `pd.read_csv()`
* SAS: `pd.read_sas()`
* Excel: `pd.read_excel()`
* Stata: `pd.read_stata()`

Whenever we look at larger dataframes, we will be using `df.head(10)` to inspect the first 10 rows, or `df.sample(10)` to look at 10 random rows (when the first 10 are special, for example). 

In [19]:
# example: raw download from DST
# note: the file must be in a sub folder "data" to the folder where jupyter was launched
filename = './data/RAS200.xlsx'

In [20]:
# first attempt
pd.read_excel(filename).head(10)

,"Erhvervs- og beskæftigelsesfrekvenser (ultimo november) efter frekvens, køn, alder, herkomst, område og tid",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,Enhed: pct.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0
2,Beskæftigelsesfrekvens,I alt,Alder i alt,I alt,Hele landet,76.0,72.8,72.1,71.8,71.3,71.3,71.6,72.1,72.8
3,NaN,NaN,NaN,NaN,Region Hovedstaden,75.8,73.3,72.6,72.5,72.2,72.3,72.5,73.2,73.8
4,NaN,NaN,NaN,NaN,Landsdel Byen København,73.4,71.0,70.5,70.5,70.4,70.6,70.9,71.6,72.3
5,NaN,NaN,NaN,NaN,København,72.5,70.0,69.7,69.6,69.4,69.7,70.0,70.8,71.5
6,NaN,NaN,NaN,NaN,Frederiksberg,75.6,73.7,72.7,72.8,73.0,73.3,73.5,74.0,74.7
7,NaN,NaN,NaN,NaN,Dragør,80.5,78.8,78.5,78.5,78.7,78.7,78.9,78.6,79.2
8,NaN,NaN,NaN,NaN,Tårnby,79.0,76.3,75.4,75.8,75.4,75.8,75.7,76.5,77.4
9,NaN,NaN,NaN,NaN,Landsdel Københavns omegn,77.1,74.2,73.2,73.0,72.4,72.4,72.6,73.3,73.8


### Getting the right columns and rows

Clearly, we should skip the first three rows and the first four columns

In [21]:
empl = pd.read_excel(filename, skiprows=2)
empl.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Beskæftigelsesfrekvens,I alt,Alder i alt,I alt,Hele landet,76.0,72.8,72.1,71.8,71.3,71.3,71.6,72.1,72.8
1,NaN,NaN,NaN,NaN,Region Hovedstaden,75.8,73.3,72.6,72.5,72.2,72.3,72.5,73.2,73.8
2,NaN,NaN,NaN,NaN,Landsdel Byen København,73.4,71.0,70.5,70.5,70.4,70.6,70.9,71.6,72.3
3,NaN,NaN,NaN,NaN,København,72.5,70.0,69.7,69.6,69.4,69.7,70.0,70.8,71.5
4,NaN,NaN,NaN,NaN,Frederiksberg,75.6,73.7,72.7,72.8,73.0,73.3,73.5,74.0,74.7
5,NaN,NaN,NaN,NaN,Dragør,80.5,78.8,78.5,78.5,78.7,78.7,78.9,78.6,79.2
6,NaN,NaN,NaN,NaN,Tårnby,79.0,76.3,75.4,75.8,75.4,75.8,75.7,76.5,77.4
7,NaN,NaN,NaN,NaN,Landsdel Københavns omegn,77.1,74.2,73.2,73.0,72.4,72.4,72.6,73.3,73.8
8,NaN,NaN,NaN,NaN,Albertslund,73.1,69.2,67.5,67.4,66.5,67.0,67.4,68.7,68.8
9,NaN,NaN,NaN,NaN,Ballerup,75.9,72.9,71.9,72.0,71.4,71.0,71.8,72.8,73.6


The first couple of columns are not needed and contain only missing values (denoted by `NaN` (Not a Number)), so we will drop those. 

In [22]:
drop_these = ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3']
empl.drop(drop_these, axis=1, inplace=True) # axis = 1 -> columns, inplace=True -> changed, no copy made
empl.head(5)

,Unnamed: 4,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Hele landet,76.0,72.8,72.1,71.8,71.3,71.3,71.6,72.1,72.8
1,Region Hovedstaden,75.8,73.3,72.6,72.5,72.2,72.3,72.5,73.2,73.8
2,Landsdel Byen København,73.4,71.0,70.5,70.5,70.4,70.6,70.9,71.6,72.3
3,København,72.5,70.0,69.7,69.6,69.4,69.7,70.0,70.8,71.5
4,Frederiksberg,75.6,73.7,72.7,72.8,73.0,73.3,73.5,74.0,74.7


> **Alternative:** Use `del empl['Unnamed: 0']`.

### Renaming variables

Let's rename the first variable, which is now called `Unnamed: 4`. This is done using `df.rename(columns=dict)`, where dict must be a Python *dictionary*.

In [23]:
empl.rename(columns = {'Unnamed: 4':'municipality'}, inplace=True)

We also see that the employment rate in 2008 has been named `2008`. Having a variable that is named a number can cause problems with some functions (and many other programming languages do not even allow it), so let us change their names. To do so, we need to create a dictionary that maps each of the years {2008, ..., 2016} to {e2008, ..., e2016}. 

In [24]:
myDict = {}
for i in range(2008, 2017): # range goes from 2008 to but not including 2017
    myDict[str(i)] = f'e{i}' 
myDict

{'2008': 'e2008',
 '2009': 'e2009',
 '2010': 'e2010',
 '2011': 'e2011',
 '2012': 'e2012',
 '2013': 'e2013',
 '2014': 'e2014',
 '2015': 'e2015',
 '2016': 'e2016'}

In [25]:
empl.rename(columns = myDict, inplace=True)
empl.head(10)

,municipality,e2008,e2009,e2010,e2011,e2012,e2013,e2014,e2015,e2016
0,Hele landet,76.0,72.8,72.1,71.8,71.3,71.3,71.6,72.1,72.8
1,Region Hovedstaden,75.8,73.3,72.6,72.5,72.2,72.3,72.5,73.2,73.8
2,Landsdel Byen København,73.4,71.0,70.5,70.5,70.4,70.6,70.9,71.6,72.3
3,København,72.5,70.0,69.7,69.6,69.4,69.7,70.0,70.8,71.5
4,Frederiksberg,75.6,73.7,72.7,72.8,73.0,73.3,73.5,74.0,74.7
5,Dragør,80.5,78.8,78.5,78.5,78.7,78.7,78.9,78.6,79.2
6,Tårnby,79.0,76.3,75.4,75.8,75.4,75.8,75.7,76.5,77.4
7,Landsdel Københavns omegn,77.1,74.2,73.2,73.0,72.4,72.4,72.6,73.3,73.8
8,Albertslund,73.1,69.2,67.5,67.4,66.5,67.0,67.4,68.7,68.8
9,Ballerup,75.9,72.9,71.9,72.0,71.4,71.0,71.8,72.8,73.6


Now we can find the employment rate in the municipality where Anders grew up: 

In [26]:
empl.loc[empl.municipality == 'Lejre']

,municipality,e2008,e2009,e2010,e2011,e2012,e2013,e2014,e2015,e2016
40,Lejre,81.5,79.4,78.3,78.2,77.5,77.4,77.3,78.2,78.6


### Dropping observations that are not actually municipalities 

The dataset contains observations like "Region Hovedstaden", which is not a municipality so we want to drop such rows. To do this, we can use the `df['var'].str` functionalities, in particular `df['var'].str.contains('PATTERN')`. 

In [27]:
I  = empl.municipality.str.contains('Region')
empl.loc[I, :]

,municipality,e2008,e2009,e2010,e2011,e2012,e2013,e2014,e2015,e2016
1,Region Hovedstaden,75.8,73.3,72.6,72.5,72.2,72.3,72.5,73.2,73.8
36,Region Sjælland,75.7,72.7,71.9,71.6,70.9,71.0,71.3,71.9,72.6
56,Region Syddanmark,75.7,72.0,71.2,70.7,70.0,70.1,70.5,70.8,71.6
81,Region Midtjylland,77.0,73.4,72.8,72.4,71.8,71.7,72.2,72.5,73.3
103,Region Nordjylland,75.2,71.5,71.2,71.0,70.6,70.2,70.3,70.7,71.7


Delete these rows. 

In [28]:
for val in ['Region', 'Landsdel', 'Hele landet']: 
    I = empl.municipality.str.contains(val)
    empl = empl.loc[I == False] # keep everything else

### Summary statistics 

To get an overview of employments across municipalities we can use the function `df.describe()`. Note that each observation (municipality) is weighted equally. 

In [29]:
empl.describe()

,e2008,e2009,e2010,e2011,e2012,e2013,e2014,e2015,e2016
count,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000
mean,76.629293,73.419192,72.794949,72.591919,72.020202,72.091919,72.431313,72.876768,73.669697
std,3.727538,3.626537,3.617062,3.691211,3.804240,3.704484,3.660062,3.647275,3.549690
min,62.900000,63.400000,62.400000,61.500000,60.700000,61.200000,61.600000,62.200000,62.600000
25%,74.650000,71.350000,70.550000,70.000000,69.450000,69.700000,70.200000,70.750000,71.500000
50%,76.800000,73.300000,72.900000,72.600000,72.100000,72.200000,72.600000,73.100000,74.000000
75%,79.150000,75.900000,75.200000,75.050000,74.500000,74.500000,74.700000,75.200000,75.700000
max,84.000000,81.500000,81.400000,80.500000,80.700000,80.500000,80.900000,81.200000,81.800000


In [30]:
empl.mean()

e2008    76.629293
e2009    73.419192
e2010    72.794949
e2011    72.591919
e2012    72.020202
e2013    72.091919
e2014    72.431313
e2015    72.876768
e2016    73.669697
dtype: float64

## Long vs. wide datasets: `pd.wide_to_long()`

Often in economic applications, it can be useful to switch between *wide* vs. *long* formats (long is sometimes referred to as *tall*, e.g. in Stata). 

This is done by the commands `pd.wide_to_long()` (and `pd.long_to_wide()`). 

Many types of analyses are easier to do in one format than in another so it is extremely useful to be able to switch comfortably between formats. 

**Common:** Think of a dataset as having an "ID" and a "PERIOD" variable. In our dataset `empl`, the ID variable is `municipality`, and the period variable is `year`. 

**Wide dataset:** The default from Statistics Denmark: each row corresponds to an ID and there is a variable for each PERIOD. 

**Tall dataset:** There is one row for each combination of (ID, PERIOD). 

In general, Pandas will assume that the variables in the *wide* format have a particular structure: namely they are of the form XPERIOD, where X is called the "stub". In our case, the variable names are e.g. `e2011`, so the stub is `e` and the period (for that variable) is `2011`. 

In [31]:
empl_tall = pd.wide_to_long(empl, stubnames='e', i='municipality', j='year')
empl_tall.head(10)

,,e
municipality,year,
København,2008,72.5
Frederiksberg,2008,75.6
Dragør,2008,80.5
Tårnby,2008,79.0
Albertslund,2008,73.1
Ballerup,2008,75.9
Brøndby,2008,72.3
Gentofte,2008,79.2
Gladsaxe,2008,77.9


**Note:** The variables `municipality` and `year`  are now in the index!! We see that because they are "below" `e` in the `head` overview. 

In [32]:
empl_tall = empl_tall.reset_index()

Now we can look for the municipality we care about

In [33]:
empl_tall.loc[empl_tall.municipality == 'Lejre', :]

,municipality,year,e
32,Lejre,2008,81.5
131,Lejre,2009,79.4
230,Lejre,2010,78.3
329,Lejre,2011,78.2
428,Lejre,2012,77.5
527,Lejre,2013,77.4
626,Lejre,2014,77.3
725,Lejre,2015,78.2
824,Lejre,2016,78.6


**Teaser:** As a quick teaser for what's to come, here's a cute little plot using the builtin pandas plot function. 

In [34]:
empl_tall.loc[empl_tall['municipality'] == 'Lejre', :].plot(x='year',y='e'); 

We can even do it interactively:

In [35]:
import ipywidgets as widgets
def plot_e(dataframe, municipality): 
    I = dataframe['municipality'] == municipality
    dataframe.loc[I,:].plot(x='year', y='e')

In [36]:
widgets.interact(plot_e, 
    dataframe = widgets.fixed(empl_tall),
    municipality = widgets.Dropdown(description='Municipality', options=empl_tall.municipality.unique(), value='Lejre')
); 

interactive(children=(Dropdown(description='Municipality', index=32, options=('København', 'Frederiksberg', 'D…

## Income

Next, we will read in the avg. disposable income for highly educated in each municipality. Here we do the cleaning, renaming and structuring in a few condensed lines.

In [37]:
# a. load
inc = pd.read_excel('./data/INDKP107.xlsx', skiprows=2)

# b. clean and rename
inc.drop([f'Unnamed: {i}' for i in range(4)], axis=1, inplace=True)
inc.rename(columns = {'Unnamed: 4':'municipality'}, inplace=True)
inc.rename(columns = {str(i): f'inc{i}' for i in range(2004,2018)}, inplace=True)

# c. drop rows with missing
inc.dropna(inplace=True)

# d. remove non-municipalities
for val in ['Region','Landsdel', 'Hele landet']: 
    I = inc.municipality.str.contains(val)
    inc.drop(inc[I].index, inplace=True)
    
inc.head(5)

,municipality,inc2004,inc2005,inc2006,inc2007,inc2008,inc2009,inc2010,inc2011,inc2012,inc2013,inc2014,inc2015,inc2016,inc2017
3,København,233204.0,244462.0,255315.0,254533.0,259216.0,266636.0,297422.0,303717.0,308601.0,316362.0,325070.0,334083.0,327179.0,333579.0
4,Frederiksberg,276111.0,291299.0,305539.0,301547.0,296988.0,304406.0,349249.0,356537.0,363466.0,376072.0,379308.0,397210.0,389151.0,409097.0
5,Dragør,326324.0,330289.0,339630.0,367640.0,371290.0,375508.0,425605.0,471854.0,457410.0,469260.0,482857.0,494313.0,475567.0,490152.0
6,Tårnby,264299.0,273414.0,291735.0,279844.0,288973.0,296610.0,336077.0,368789.0,347705.0,357150.0,359455.0,372565.0,360230.0,373304.0
8,Albertslund,253319.0,260060.0,270290.0,267863.0,269855.0,283067.0,313767.0,308977.0,320298.0,328004.0,334630.0,344447.0,334094.0,332943.0


Convert wide -> tall:

In [38]:
inc_tall = pd.wide_to_long(df=inc, stubnames='inc', i='municipality', j='year').reset_index()
inc_tall.head(5)

,municipality,year,inc
0,København,2004,233204.0
1,Frederiksberg,2004,276111.0
2,Dragør,2004,326324.0
3,Tårnby,2004,264299.0
4,Albertslund,2004,253319.0


## Municipal area

Finally, let's read in a dataset on municipality areas in km$^2$. 

In [39]:
# a. load
area = pd.read_excel('./data/areal.xlsx', skiprows=2)

# b. clean and rename
area.rename(columns = {'Unnamed: 0':'municipality','areal':'km2'}, inplace=True)

# c. drop rows with missing
area.dropna(inplace=True)

# d. remove non-municipalities
for val in ['Region','Landsdel', 'Hele landet']: 
    I = area.municipality.str.contains(val)
    area.drop(area[I].index, inplace=True)
    
area.head(5)

,municipality,km2
2,København,86.6
3,Frederiksberg,8.7
4,Dragør,18.3
5,Tårnby,66.1
6,Albertslund,23.2


## Writing data

As with reading in data, we have the corresponding functions:
* df.to_csv()
* df.to_excel()
* df.to_stata()
* df.to_sas()
* df.to_hdf(): Pro = low memory footprint, Cons = restrictive as to which variables you can save; Python only. 
* df.to_pickle(): Pro = quick and dirty, can save all variable types. Cons = Python only; quite unsafe (a malicious hacker can store a virus in a pickle!). 

Let's save our dataset to CSV form. We will set `index=False` to avoid saving the index (which does not mean anything here but can in other contexts be an annoying thing). 

In [40]:
empl_tall.to_csv('./data/RAS200_tall.csv', index=False)

In [41]:
inc_tall.to_csv('./data/INDKP107_tall.csv', index=False)

In [42]:
area.to_csv('./data/area.csv', index=False)